# Preliminary Results

Import libraries

In [290]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
import matplotlib
import matplotlib.pyplot as plt
import math
import sklearn
from sklearn.dummy import DummyClassifier as Dummy
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC
%matplotlib inline

In [354]:
movie_data = pd.read_csv('movie_metadata.csv')
movie_data = movie_data.dropna()
movie_data = movie_data[movie_data['movie_facebook_likes'] > 0]

fig = plt.figure(figsize=(20,15))

# create genre list
genre_list = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'History', 'Horror', 'Mystery', 'Romance']

# create categorical variables for these common genres
for genre in genre_list:
    lst = []
    for index, row in movie_data.iterrows():
        if (row['genres']).find(genre) != -1:
            lst.append(1)
        else:
            lst.append(0)
    movie_data[genre] = lst

In [291]:
# create dataframe for testing
train_test = pd.DataFrame({})

# add relevent variables
train_test = pd.concat([train_test, movie_data['director_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['cast_total_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['budget']], axis = 1)
train_test = pd.concat([train_test, movie_data['imdb_score']], axis = 1)

# split variables
training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)

# create linear model to fit and score
lr_model = LinReg()
lr_model.fit(training.drop('imdb_score', axis=1), training['imdb_score'])

print "Naive Linear Regression Model", lr_model.score(testing.drop('imdb_score', axis = 1), testing['imdb_score'])

Naive Linear Regression Model 0.0805890115215


In [355]:
### try using genre categories

train_test = pd.DataFrame({})

for genre in genre_list:
    train_test = pd.concat([train_test, movie_data[genre]], axis = 1)

train_test = pd.concat([train_test, movie_data['imdb_score']], axis = 1)
training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)

lr_model = LinReg()
lr_model.fit(training.drop('imdb_score', axis=1), training['imdb_score'])

print "Naive Linear Regression Model",lr_model.score(testing.drop('imdb_score', axis = 1), testing['imdb_score'])

Naive Linear Regression Model 0.157014670481


In [118]:
### try all variables we've tested so far

train_test = pd.DataFrame({})

for genre in genre_list:
    train_test = pd.concat([train_test, movie_data[genre]], axis = 1)

train_test = pd.concat([train_test, movie_data['director_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['cast_total_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['budget']], axis = 1)
train_test = pd.concat([train_test, movie_data['imdb_score']], axis = 1)

training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)


lr_model = LinReg()
lr_model.fit(training.drop('imdb_score', axis=1), training['imdb_score'])

print "Naive Linear Regression Model", lr_model.score(testing.drop('imdb_score', axis = 1), testing['imdb_score'])

Naive Linear Regression Model 0.218270863766


In [325]:
### predict gross box office

train_test = pd.DataFrame({})

for genre in genre_list:
    train_test = pd.concat([train_test, movie_data[genre]], axis = 1)

train_test = pd.concat([train_test, movie_data['director_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['cast_total_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['budget']], axis = 1)
train_test = pd.concat([train_test, movie_data['gross']], axis = 1)

training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)


lr_model = LinReg()
lr_model.fit(training.drop('gross', axis=1), training['gross'])

print "Naive Linear Regression Model", lr_model.score(testing.drop('gross', axis = 1), testing['gross'])

Naive Linear Regression Model 0.206170001658


In [326]:
# create classes for score
lst = []
for index, row in movie_data.iterrows():
        if row['imdb_score'] > 8.5:
            lst.append(5)
        elif row['imdb_score'] > 8.0:
            lst.append(4)
        elif row['imdb_score'] > 7.0:
            lst.append(3)
        elif row['imdb_score'] > 5.0: 
            lst.append(2)
        else:
            lst.append(1)

movie_data['quality_class'] = lst

In [327]:
# create classes for facebook likes
lst = []
for index, row in movie_data.iterrows():
        if row['movie_facebook_likes'] > 150000:
            lst.append(5)
        elif row['movie_facebook_likes'] > 80000:
            lst.append(4)
        elif row['movie_facebook_likes'] > 20000:
            lst.append(3)
        elif row['movie_facebook_likes'] > 7500: 
            lst.append(2)
        else:
            lst.append(1)

movie_data['popularity_class'] = lst

In [343]:
# create dataframe for variables
train_test = pd.DataFrame({})

for genre in genre_list:
    train_test = pd.concat([train_test, movie_data[genre]], axis = 1)

train_test = pd.concat([train_test, movie_data['director_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['cast_total_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['quality_class']], axis = 1)

# split data
training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)

In [344]:
# score SVC model for imdb class
print "IMDB score class"
d_model = Dummy()
d_model.fit(training.drop('quality_class', axis=1), training['quality_class'])
print "\tDummy Model", d_model.score(testing.drop('quality_class', axis = 1), testing['quality_class'])

svc_model = SVC(C= 100)
svc_model.fit(training.drop('quality_class', axis=1), training['quality_class'])

print "\tNaive SVC Model", lr_model.score(testing.drop('quality_class', axis = 1), testing['quality_class'])

IMDB score class
	Dummy Model 0.424317617866
	Naive SVC Model 0.572373862696


In [345]:
# score SVC model for facebook likes class (with budget)
train_test = pd.DataFrame({})

for genre in genre_list:
    train_test = pd.concat([train_test, movie_data[genre]], axis = 1)

train_test = pd.concat([train_test, movie_data['director_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['cast_total_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['budget']], axis = 1)
train_test = pd.concat([train_test, movie_data['popularity_class']], axis = 1)

training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)

In [346]:
print "Popularity class (with Budget)"
d_model = Dummy()
d_model.fit(training.drop('popularity_class', axis=1), training['popularity_class'])
print "\tDummy Model", d_model.score(testing.drop('popularity_class', axis = 1), testing['popularity_class'])

svc_model = SVC(C= 100)
svc_model.fit(training.drop('popularity_class', axis=1), training['popularity_class'])

print "\tNaive SVC Model", svc_model.score(testing.drop('popularity_class', axis = 1), testing['popularity_class'])

Popularity class (with Budget)
	Dummy Model 0.342431761787
	Naive SVC Model 0.471464019851


In [347]:
# remove budget variable
train_test = train_test.drop('budget', axis = 1)
training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)

In [348]:
# score SVC model for facebook likes class (without budget)
print "Popularity class (without Budget)"
d_model = Dummy()
d_model.fit(training.drop('popularity_class', axis=1), training['popularity_class'])
print "Dummy Model", d_model.score(testing.drop('popularity_class', axis = 1), testing['popularity_class'])

svc_model = SVC()
svc_model.fit(training.drop('popularity_class', axis=1), training['popularity_class'])
print "Naive SVC Model", svc_model.score(testing.drop('popularity_class', axis = 1), testing['popularity_class'])

Popularity class (without Budget)
Dummy Model 0.344086021505
Naive SVC Model 0.502894954508


In [349]:
# create classes for gross box office
lst = []
for index, row in movie_data.iterrows():
        if row['gross'] > 600000000:
            lst.append(5)
        elif row['gross'] > 200000000:
            lst.append(4)
        elif row['gross'] > 50000000:
            lst.append(3)
        elif row['gross'] > 10000000: 
            lst.append(2)
        else:
            lst.append(1)

movie_data['gross_class'] = lst

In [350]:
# create dataframe of relevent variables
train_test = pd.DataFrame({})

for genre in genre_list:
    train_test = pd.concat([train_test, movie_data[genre]], axis = 1)

train_test = pd.concat([train_test, movie_data['director_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['cast_total_facebook_likes']], axis = 1)
train_test = pd.concat([train_test, movie_data['budget']], axis = 1)
train_test = pd.concat([train_test, movie_data['gross_class']], axis = 1)

training, testing = sklearn.model_selection.train_test_split(train_test, train_size = 0.4, test_size = 0.6)

In [351]:
# score SVC model for gross box office class
print "Box Office class (unadjusted)"
d_model = Dummy()
d_model.fit(training.drop('gross_class', axis=1), training['gross_class'])
print "\tDummy Model", d_model.score(testing.drop('gross_class', axis = 1), testing['gross_class'])

svc_model = SVC(C=100)
svc_model.fit(training.drop('gross_class', axis=1), training['gross_class'])
print "\tNaive SVC Model", svc_model.score(testing.drop('gross_class', axis = 1), testing['gross_class'])

Box Office class (unadjusted)
	Dummy Model 0.285359801489
	Naive SVC Model 0.39123242349
